Deep Learning
=============

Assignment 3
------------

Previously in `2_fullyconnected.ipynb`, you trained a logistic regression and a neural network model.

The goal of this assignment is to explore regularization techniques.

In [1]:
# These are all the modules we'll be using later. Make sure you can import them
# before proceeding further.
from __future__ import print_function
import numpy as np
import tensorflow as tf
from six.moves import cPickle as pickle

First reload the data we generated in _notmist.ipynb_.

In [2]:
pickle_file = 'notMNIST.pickle'

with open(pickle_file, 'rb') as f:
    save = pickle.load(f)
    train_dataset = save['train_dataset']
    train_labels = save['train_labels']
    valid_dataset = save['valid_dataset']
    valid_labels = save['valid_labels']
    test_dataset = save['test_dataset']
    test_labels = save['test_labels']
    del save  # hint to help gc free up memory
    
    print('Training set', train_dataset.shape, train_labels.shape)
    print('Validation set', valid_dataset.shape, valid_labels.shape)
    print('Test set', test_dataset.shape, test_labels.shape)

Training set (200000, 28, 28) (200000,)
Validation set (10000, 28, 28) (10000,)
Test set (10000, 28, 28) (10000,)


Reformat into a shape that's more adapted to the models we're going to train:
- data as a flat matrix,
- labels as float 1-hot encodings.

In [3]:
image_size = 28
num_labels = 10

def reformat(dataset, labels):
    dataset = dataset.reshape((-1, image_size * image_size)).astype(np.float32)
    # Map 2 to [0.0, 1.0, 0.0 ...], 3 to [0.0, 0.0, 1.0 ...]
    labels = (np.arange(num_labels) == labels[:,None]).astype(np.float32)
    return dataset, labels

train_dataset, train_labels = reformat(train_dataset, train_labels)
valid_dataset, valid_labels = reformat(valid_dataset, valid_labels)
test_dataset, test_labels = reformat(test_dataset, test_labels)

print('Training set', train_dataset.shape, train_labels.shape)
print('Validation set', valid_dataset.shape, valid_labels.shape)
print('Test set', test_dataset.shape, test_labels.shape)

Training set (200000, 784) (200000, 10)
Validation set (10000, 784) (10000, 10)
Test set (10000, 784) (10000, 10)


In [4]:
def accuracy(predictions, labels):
    return (100.0 * np.sum(np.argmax(predictions, 1) == np.argmax(labels, 1)) / predictions.shape[0])

---
Problem 1
---------

Introduce and tune L2 regularization for both logistic and neural network models. Remember that L2 amounts to adding a penalty on the norm of the weights to the loss. In TensorFlow, you can compute the L2 loss for a tensor `t` using `nn.l2_loss(t)`. The right amount of regularization should improve your validation / test accuracy.

---

In [61]:
batch_size = 128
hidden_level_size = 1024

graph = tf.Graph()
with graph.as_default():

    tf_train_dataset = tf.placeholder(tf.float32,
                                      shape=(batch_size, image_size * image_size))
    tf_train_labels = tf.placeholder(tf.float32, shape=(batch_size, num_labels))
    tf_valid_dataset = tf.constant(valid_dataset)
    tf_test_dataset = tf.constant(test_dataset)
  
    w1 = tf.Variable(tf.truncated_normal([image_size * image_size, hidden_level_size]))
    b1 = tf.Variable(tf.zeros([hidden_level_size]))

    w2 = tf.Variable(tf.truncated_normal([hidden_level_size, num_labels]))
    b2 = tf.Variable(tf.zeros([num_labels]))
    
    # 2 layer :)
    logits = tf.matmul(tf.nn.relu(tf.matmul(tf_train_dataset, w1) + b1), w2) + b2
    
    # original loss
    #loss = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits(logits, tf_train_labels))
    # -> 88%;89%;89.3%;89.1%
    #
    # new loss with L2 regularization
    #  * ha csak w1-re L2-zünk: 0.01 -> 91%;89%;90.7%;92.0%! :)
    #    (0.5 -> 71% ; 0.1 -> 78% ; 0.05 -> 86%)
    #  * ha w1 és w2-re is L2: 0.05;0.05 -> 91.6%;91.6%;91.7%! :)
    #    (0.01;0.01 -> 90.4%;90.2%)
    l2_weight_beta_w1 = 0.005
    l2_weight_beta_w2 = 0.005
    loss = (
        tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits(logits,tf_train_labels))
        + l2_weight_beta_w1 * tf.nn.l2_loss(w1)
        + l2_weight_beta_w2 * tf.nn.l2_loss(w2)
    )
    # XXX meta-optimalizálót csinálni az l2_weight_beta -ra hogy lehetne? :)
    
    optimizer = tf.train.GradientDescentOptimizer(0.5).minimize(loss)
  
    train_prediction = tf.nn.softmax(logits)
    valid_prediction = tf.nn.softmax(tf.matmul(tf.nn.relu(tf.matmul(tf_valid_dataset, w1) + b1), w2) + b2)
    test_prediction = tf.nn.softmax(tf.matmul(tf.nn.relu(tf.matmul(tf_test_dataset, w1) + b1), w2) + b2)

In [62]:
num_steps = 3001
#num_steps = 4

with tf.Session(graph=graph) as session:

    tf.initialize_all_variables().run()
    print("Initialized\n")

    for step in range(num_steps):

        offset = (step * batch_size) % (train_labels.shape[0] - batch_size)
        batch_data = train_dataset[offset:(offset + batch_size), :]
        batch_labels = train_labels[offset:(offset + batch_size), :]

        feed_dict = {tf_train_dataset : batch_data, tf_train_labels : batch_labels}
    
        session.run(optimizer, feed_dict=feed_dict)
        
        if (step % 500 == 0):
            print("Minibatch loss at step %d: %f" % (step, loss.eval(feed_dict=feed_dict)))
            print("Minibatch accuracy: %.1f%%" % accuracy(train_prediction.eval(feed_dict=feed_dict), batch_labels))
            print("Validation accuracy: %.1f%%\n" % accuracy(valid_prediction.eval(), valid_labels))
    print("Test accuracy: %.1f%%" % accuracy(test_prediction.eval(), test_labels))

Initialized

Minibatch loss at step 0: 2655.076172
Minibatch accuracy: 43.8%
Validation accuracy: 35.2%

Test accuracy: 53.7%


---
Problem 2
---------
Let's demonstrate an extreme case of overfitting. Restrict your training data to just a few batches. What happens?

**Result: ** accuracy drops.
If `num_steps = 4` -> accuracy = 59.5%. :)

---

---
Problem 3
---------
Introduce Dropout on the hidden layer of the neural network. Remember: Dropout should only be introduced during training, not evaluation, otherwise your evaluation results would be stochastic as well. TensorFlow provides `nn.dropout()` for that, but you have to make sure it's only inserted during training.

What happens to our extreme overfitting case?

**Result:** Hm.. it remained nearly the same with `num_steps = 4`...

---

In [84]:
batch_size = 128
#hidden_level_size = 1024
#hidden_level_size = 1536
hidden_level_size = 512

graph = tf.Graph()
with graph.as_default():

    tf_train_dataset = tf.placeholder(tf.float32,
                                      shape=(batch_size, image_size * image_size))
    tf_train_labels = tf.placeholder(tf.float32, shape=(batch_size, num_labels))
    tf_valid_dataset = tf.constant(valid_dataset)
    tf_test_dataset = tf.constant(test_dataset)
  
    w1 = tf.Variable(tf.truncated_normal([image_size * image_size, hidden_level_size]))
    b1 = tf.Variable(tf.zeros([hidden_level_size]))

    w2 = tf.Variable(tf.truncated_normal([hidden_level_size, num_labels]))
    b2 = tf.Variable(tf.zeros([num_labels]))
    
    # 2 layer :)
    # original logits (+ L2) - dropout nélkül: 91.6% (ld. alább)
    #logits = tf.matmul(tf.nn.relu(tf.matmul(tf_train_dataset, w1) + b1), w2) + b2
    #
    # dropout bevezetése ennyi: relu után egy dropout :)
    #
    #  * hidden_layer_size=1024 + L2(0.05;0.05) + dropout: 91.2%;91.3% hm.. nem javít
    #  * hidden_layer_size=1024 + dropout: 87.6%;87.2% hm.. nem javít
    #
    #  * hidden_layer_size=1536: 88.7%;89.6% -- hát, kicsit tán javított ez? hátha a dropout nagyon! :)
    #  * hidden_layer_size=1536 + dropout: 88.1%;88.2% hm.. a dropout itt is ront..
    # 
    #  * hidden_layer_size=512: 87.8%;87.8% -- persze, nem javított :)
    #  * hidden_layer_size=512 + dropout: 86.2%;86.4% hm.. a dropout itt is ront..
    dropout_keep_prob = 0.5
    logits = tf.matmul(tf.nn.dropout(tf.nn.relu(tf.matmul(tf_train_dataset, w1) + b1), dropout_keep_prob), w2) + b2
    
    # original loss
    loss = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits(logits, tf_train_labels))
    # -> 88%;89%;89.3%;89.1%
    #
    # new loss with L2 regularization
    #  * ha csak w1-re L2-zünk: 0.01 -> 91%;89%;90.7%;92.0%! :)
    #    (0.5 -> 71% ; 0.1 -> 78% ; 0.05 -> 86%)
    #  * ha w1 és w2-re is L2: 0.05;0.05 -> 91.6%;91.6%;91.7%! :)
    #    (0.01;0.01 -> 90.4%;90.2%)
    #l2_weight_beta_w1 = 0.005
    #l2_weight_beta_w2 = 0.005
    #loss = (
    #    tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits(logits,tf_train_labels))
    #    + l2_weight_beta_w1 * tf.nn.l2_loss(w1)
    #    + l2_weight_beta_w2 * tf.nn.l2_loss(w2)
    #)
    
    optimizer = tf.train.GradientDescentOptimizer(0.5).minimize(loss)
  
    train_prediction = tf.nn.softmax(logits)
    valid_prediction = tf.nn.softmax(tf.matmul(tf.nn.relu(tf.matmul(tf_valid_dataset, w1) + b1), w2) + b2)
    test_prediction = tf.nn.softmax(tf.matmul(tf.nn.relu(tf.matmul(tf_test_dataset, w1) + b1), w2) + b2)

In [85]:
num_steps = 3001
#num_steps = 4

with tf.Session(graph=graph) as session:

    tf.initialize_all_variables().run()
    print("Initialized\n")

    for step in range(num_steps):

        offset = (step * batch_size) % (train_labels.shape[0] - batch_size)
        batch_data = train_dataset[offset:(offset + batch_size), :]
        batch_labels = train_labels[offset:(offset + batch_size), :]

        feed_dict = {tf_train_dataset : batch_data, tf_train_labels : batch_labels}
    
        session.run(optimizer, feed_dict=feed_dict)
        
        if (step % 500 == 0):
            print("Minibatch loss at step %d: %f" % (step, loss.eval(feed_dict=feed_dict)))
            print("Minibatch accuracy: %.1f%%" % accuracy(train_prediction.eval(feed_dict=feed_dict), batch_labels))
            print("Validation accuracy: %.1f%%\n" % accuracy(valid_prediction.eval(), valid_labels))
    print("Test accuracy: %.1f%%" % accuracy(test_prediction.eval(), test_labels))

Initialized

Minibatch loss at step 0: 372.573517
Minibatch accuracy: 49.2%
Validation accuracy: 34.3%

Minibatch loss at step 500: 8.367976
Minibatch accuracy: 73.4%
Validation accuracy: 73.7%

Minibatch loss at step 1000: 2.482793
Minibatch accuracy: 68.8%
Validation accuracy: 75.2%

Minibatch loss at step 1500: 3.408662
Minibatch accuracy: 76.6%
Validation accuracy: 77.5%

Minibatch loss at step 2000: 3.594660
Minibatch accuracy: 75.0%
Validation accuracy: 78.6%

Minibatch loss at step 2500: 2.072365
Minibatch accuracy: 80.5%
Validation accuracy: 78.4%

Minibatch loss at step 3000: 1.261629
Minibatch accuracy: 70.3%
Validation accuracy: 79.0%

Test accuracy: 86.4%


---
Problem 4
---------

Try to get the best performance you can using a multi-layer model! The best reported test accuracy using a deep network is [97.1%](http://yaroslavvb.blogspot.com/2011/09/notmnist-dataset.html?showComment=1391023266211#c8758720086795711595).

One avenue you can explore is to add multiple layers.

Another one is to use learning rate decay:

    global_step = tf.Variable(0)  # count the number of steps taken.
    learning_rate = tf.train.exponential_decay(0.5, global_step, ...)
    optimizer = tf.train.GradientDescentOptimizer(learning_rate).minimize(loss, global_step=global_step)
 
 **Utóbbi egyelőre elmaradt.**
 
 ---


In [13]:
batch_size = 128
hidden_level_size_1 = 1024
hidden_level_size_2 = 1024

graph = tf.Graph()
with graph.as_default():

    tf_train_dataset = tf.placeholder(tf.float32, shape=(batch_size, image_size * image_size))
    tf_train_labels = tf.placeholder(tf.float32, shape=(batch_size, num_labels))
    tf_valid_dataset = tf.constant(valid_dataset)
    tf_test_dataset = tf.constant(test_dataset)
  
    w1 = tf.Variable(tf.truncated_normal([image_size * image_size, hidden_level_size_1]))
    b1 = tf.Variable(tf.zeros([hidden_level_size_1]))
    
    w2 = tf.Variable(tf.truncated_normal([hidden_level_size_1, hidden_level_size_2]))
    b2 = tf.Variable(tf.zeros([hidden_level_size_2]))
    
    w3 = tf.Variable(tf.truncated_normal([hidden_level_size_2, num_labels]))
    b3 = tf.Variable(tf.zeros([num_labels]))
    
    # alap, mert ez volt eddig 2 layerrel a legjobb: L2(0.05,0.05); dropout nélkül: 91.6%
    #                    logits = tf.matmul(tf.nn.relu(tf.matmul(tf_train_dataset, w1) + b1), w2) + b2
    
    # na, akkor legyen 3 layer -- ez még nagy nehezen megy kézzel, de nem valami áttekinthető :)
    logits = tf.matmul(tf.nn.relu(tf.matmul(tf.nn.relu(tf.matmul(tf_train_dataset, w1) + b1), w2) + b2), w3) + b3
    #
    # dropout bevezetése ennyi: relu után egy dropout :)
    #
    #dropout_keep_prob = 0.5
    # XXX ez még átírandó 3 layerre!
    #XXXlogits = tf.matmul(tf.nn.dropout(tf.nn.relu(tf.matmul(tf_train_dataset, w1) + b1), dropout_keep_prob), w2) + b2
    
    # original loss
    #loss = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits(logits, tf_train_labels))
    #
    # new loss with L2 regularization
    l2_weight_beta_w1 = 0.003
    l2_weight_beta_w2 = 0.003
    l2_weight_beta_w3 = 0.003
    loss = (
        tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits(logits,tf_train_labels))
        + l2_weight_beta_w1 * tf.nn.l2_loss(w1)
        + l2_weight_beta_w2 * tf.nn.l2_loss(w2)
        + l2_weight_beta_w3 * tf.nn.l2_loss(w3)
    )
    
    # XXX hátha a learning rate segít...
    #  * 0.5 ; 0.1: 10% -- azaz ez így "elszáll", miért, mi történik, miért pontosan 10%, mindenre ua-t mondja? miért?
    #  * 0.05: -> 76.7%;75.4%;75.3%
    #  * 0.05 és 9000 lépés: 89.7%
    #  * 0.05 és 18000 lépés: 92.5%
    #  * 0.05 és 27000 lépés: 93.8% -- wow! :)

    optimizer = tf.train.GradientDescentOptimizer(0.05).minimize(loss)
  
    train_prediction = tf.nn.softmax(logits)
    # itt is nyilván 3 rétegre kell!
    valid_prediction = tf.nn.softmax(
        tf.matmul(tf.nn.relu(tf.matmul(tf.nn.relu(tf.matmul(tf_valid_dataset, w1) + b1), w2) + b2), w3) + b3)
    test_prediction = tf.nn.softmax(
        tf.matmul(tf.nn.relu(tf.matmul(tf.nn.relu(tf.matmul(tf_test_dataset, w1) + b1), w2) + b2), w3) + b3)

In [14]:
num_steps = 27001
#num_steps = 3001
#num_steps = 4

with tf.Session(graph=graph) as session:

    tf.initialize_all_variables().run()
    print("Initialized\n")

    for step in range(num_steps):

        offset = (step * batch_size) % (train_labels.shape[0] - batch_size)
        batch_data = train_dataset[offset:(offset + batch_size), :]
        batch_labels = train_labels[offset:(offset + batch_size), :]

        feed_dict = {tf_train_dataset : batch_data, tf_train_labels : batch_labels}
    
        session.run(optimizer, feed_dict=feed_dict)
        
        if (step % 500 == 0):
            print("Minibatch loss at step %d: %f" % (step, loss.eval(feed_dict=feed_dict)))
            print("Minibatch accuracy: %.1f%%" % accuracy(train_prediction.eval(feed_dict=feed_dict), batch_labels))
            print("Validation accuracy: %.1f%%\n" % accuracy(valid_prediction.eval(), valid_labels))
    print("Test accuracy: %.1f%%" % accuracy(test_prediction.eval(), test_labels))

Initialized

Minibatch loss at step 0: 176288.640625
Minibatch accuracy: 28.9%
Validation accuracy: 23.3%

Minibatch loss at step 500: 2403.145508
Minibatch accuracy: 52.3%
Validation accuracy: 48.7%

Minibatch loss at step 1000: 2068.388184
Minibatch accuracy: 57.8%
Validation accuracy: 52.2%

Minibatch loss at step 1500: 1780.012451
Minibatch accuracy: 61.7%
Validation accuracy: 55.3%

Minibatch loss at step 2000: 1532.169922
Minibatch accuracy: 57.8%
Validation accuracy: 60.8%

Minibatch loss at step 2500: 1318.652222
Minibatch accuracy: 72.7%
Validation accuracy: 62.5%

Minibatch loss at step 3000: 1135.148193
Minibatch accuracy: 67.2%
Validation accuracy: 65.8%

Minibatch loss at step 3500: 977.074829
Minibatch accuracy: 66.4%
Validation accuracy: 68.9%

Minibatch loss at step 4000: 841.372070
Minibatch accuracy: 71.1%
Validation accuracy: 69.6%

Minibatch loss at step 4500: 723.855652
Minibatch accuracy: 75.8%
Validation accuracy: 72.1%

Minibatch loss at step 5000: 623.270996
Mi